In [20]:
import pandas as pd
import numpy as np
import quandl
from google.cloud import bigquery
from google.cloud import storage
import pickle
import datetime
import os

from ficc.utils.diff_in_days import diff_in_days_two_dates
from ficc.utils.auxiliary_variables import NUM_OF_DAYS_IN_YEAR
from ficc.utils.gcp_storage_functions import upload_data, download_data

In [24]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jupyter/ahmad_creds.json"
storage_client = storage.Client()
bq_client = bigquery.Client()

In [3]:
quandl.ApiConfig.api_key = 'C6tWjxHm29zz7L5BLQxW'

In [4]:
import gcsfs
fs = gcsfs.GCSFileSystem(project='eng-reactor-287421')
with fs.open('ficc_training_data_latest/processed_data_2022-10-21-17:41-flags.pkl') as f:
    data = pd.read_pickle(f)

In [5]:
corporate_yield = quandl.get("USTREASURY/HQMYC")
corporate_maturities = np.array(corporate_yield.columns).astype(float)

In [6]:
# corporate_yield.iloc[corporate_yield.index.get_loc(datetime.datetime(2022,9,18),method='pad')]
def get_corporate_spread(row):
    duration =  diff_in_days_two_dates(row['calc_date'],row['settlement_date'])/NUM_OF_DAYS_IN_YEAR
    maturity = min(corporate_maturities, key=lambda x:abs(x-duration))
    temp_corporate_yield = corporate_yield.iloc[corporate_yield.index.get_loc(row['trade_date'],method='pad')]
    c_yield = temp_corporate_yield[str(maturity)]
    return c_yield

In [7]:
data['corporate_yield'] = data[['trade_date','settlement_date','calc_date']].parallel_apply(get_corporate_spread,axis=1)

In [9]:
data['corporate_spread'] = (data['ficc_ycl'] - data['corporate_yield']*100)

In [22]:
def sqltodf(sql, bq_client):
    bqr = bq_client.query(sql).result()
    return bqr.to_dataframe()

In [34]:
query = '''SELECT * FROM `eng-reactor-287421.treasury_yield.daily_yield_rate` order by Date desc;'''
treasury_rate = sqltodf(query, bq_client)
treasury_rate.set_index("Date", drop=True, inplace=True)
treasury_rate = treasury_rate.transpose().to_dict()

In [38]:
def current_treasury_rate(trade):
    treasury_maturities = np.array([1,2,3,5,7,10,20,30])
    time_to_maturity = diff_in_days_two_dates(trade['last_calc_date'],trade['trade_date'])/NUM_OF_DAYS_IN_YEAR
    maturity = min(treasury_maturities, key=lambda x:abs(x-time_to_maturity))
    maturity = 'year_'+str(maturity)
    t_rate = treasury_rate[trade['trade_date'].date()][maturity]
    return t_rate

In [41]:
data['treasury_rate']

0          4.47
1          4.00
2          4.25
3          4.26
4          4.26
           ... 
5296713    1.72
5296714    2.55
5296715    2.55
5296716    2.55
5296717    2.55
Name: treasury_rate, Length: 5271064, dtype: float64

In [42]:
data['treasury_rate'] = data[['trade_date','calc_date','last_calc_date','settlement_date']].parallel_apply(current_treasury_rate, axis=1)

In [45]:
data['ficc_treasury_spread'] = data['ficc_ycl'] - (data['treasury_rate'] * 100)

In [46]:
data.to_pickle('processed_data_2022-10-21-17:41-flags-corporate-spreads.pkl')

In [47]:
upload_data(storage_client, 'ficc_training_data_latest','processed_data_2022-10-21-17:41-flags-corporate-spreads.pkl')

File processed_data_2022-10-21-17:41-flags-corporate-spreads.pkl uploaded to ficc_training_data_latest.
